<a href="https://colab.research.google.com/github/snburgos/lecturasIIC3633-2021/blob/main/Pr%C3%A1cticos/Semana%202/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**: Sebastián Burgos Martínez

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

Slope One funciona en base a las diferencias de ratings entre pares de items. Suponiendo que se quiere obtener la predicción del item B a partir del item A para un usuario se realiza lo siguiente: en el caso de tener varios usuarios se sacan las diferencias entre el item A e item B para todos los usuarios que hayan evaluado ambos items. Luego, se obtiene un promedio de las diferencias y a partir del rating del item A del usuario, se le suma la diferencia obtenida anteriormente. Ahora, esto puede escalar a ocupar más diferencias de items y ponderar según el número de diferencias en cada caso.

* Este recomendador en un algoritmo de filtrado colaborativo, porque se ocupa la información de otros usuarios con sus respectivos ratings para los items, para predecir un rating.

* Está basado en los items. Esto, debido a que compara items con items (la diferencia de los ratings entre estos). Además, el algoritmo funciona mejor si se tienen items similares o con tendencias similares y con una gran cantidad de usuarios.

* Recibe usuarios, items y ratings. Lo que hace es calcular la diferencia de rating entre los distintos items para aquellos usuarios que han evaluado estes items. Entonces, obtiene la suma de las diferencias entre 2 items, para todos los items, con su respectiva cuenta para realizar las ponderaciones.

* Al agregar un nuevo rating a la base de datos se pueden sumar las diferencias entre el nuevo rating y los otros ratings del mismo usuario a las sumatorias correspondientes. Por ejemplo, si están los items A, B y C donde el usuario había evaluado A y B y ahora se agrega el rating de C, entonces a $dev_{A,C}$ se le suma la diferencia de ratings entre A y C y a $dev_{B,C}$ se le suma la diferencia de rating entre B y C. Luego, se pueden realizar los cálculos respectivos para las predicciones con las nuevas sumatorias y cuentas.

* Como este algoritmo toma la información de muchos usuarios, el tener pocos ratings no es tan crucial. De hecho, si se tiene solamente un rating y la tendencia entre dos items es muy clara, lo más probable es que la predicción sea bastante certera. Ahora, obviamente el tener pocos ratings dificulta el trabajo, ya que, normalmente, entre más información mejor. Lo importante es que debido a la naturaleza del algoritmo, es robusto frente al tener pocos ratings, pero a medida que se agregan ratings las predicciones deberían mejorar.

# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    338      0 --:--:--  0:00:01 --:--:--   338
100 1546k  100 1546k    0     0   985k      0  0:00:01  0:00:01 --:--:--  985k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    512      0 --:--:-- --:--:-- --:--:--   511
100  385k  100  385k    0     0   308k      0  0:00:01  0:00:01 --:--:-- 2991k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    544      0 --:--:-- --:--:-- --:--:--   544
100  230k  100  230k    0     0   210k      0  0:00:01  0:00:01 --:--:--  625k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 6.4 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos?

**Respuesta:**

Lo que hace `train()` es realizar los cálculos mencionados en la actividad 1. Esto es, calcular las sumas de las diferencias entre los distintos items y la cuenta de estas. Así, se tiene la información procesada para luego solo ocupar las diferencias y sus sumas.

## Paso 6:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [11]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [12]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?

In [22]:
ratings = train_file[train_file['userid'] == 457]['rating']
sum(ratings) / len(ratings)

4.038461538461538

**Respuesta:**

Estos resultados se explican, ya que la predicción se hace en base a la diferencia de items de muchos otros usuarios y se toma como base al usuario mismo. Este usuario tiene un rating promedio de 4.04. Entonces más o menos esa es su base (ya que probablemente no se cuenta el item 9 o item 1168 respectivamente). Luego, a esta base se le suma la diferencia que en un caso es 0.49 y el otro 0.13 aproximadamente. Así, se obtienen los resultados. Como en usuario dio un rating de 5 para ambos casos, estas predicciones son bastante efectivas, ya que para el id 9, si se aproxima, da 5, mientras que para el id 1168, aunque no da tan cerca de 5, si se acerca a partir de la base. O sea, en ambos casos la predicción se mejora al sumar la diferencia en comparación a la base sola.

In [13]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [23]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [24]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:**

Existe una clara diferencia. De las 5 recomendaciones solamente hay una que coincide y, está en una posición distinta. Que existan igualdades tiene mucho sentido, ya que se buscan las diferencias positivas más grandes para recomendar. Entonces, si una película es buena, esta diferencia en más grande con la mayoría de las películas. Ahora, también tiene sentido que hayan diferencias, ya que las películas que ha visto un usuario son distintas a las que ha visto otro usuario.

In [26]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:5
reclist_slopeone = mySlopeOne.recommend("5", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1080', '1653', '1512', '1536', '1463']


In [27]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1080                           Celestial Clockwork (1994)
1653    Entertaining Angels: The Dorothy Day Story (1996)
1512               World of Apu, The (Apur Sansar) (1959)
1536                                 Aiqing wansui (1994)
1463                                     Boys, Les (1997)
Name: title, dtype: object

A continuación hago el mismo experimento, pero con el top 10.

In [29]:
# Mediante el método recommend() genereremos una lista top-10 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 10)

# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1651                         Spanish Prisoner, The (1997)
1650                              Butcher Boy, The (1998)
1645                              Butcher Boy, The (1998)
1643                                    Angel Baby (1995)
Name: title, dtype: object

In [30]:
# Mediante el método recommend() genereremos una lista top-10 recomendaciones para el usuario ID:5
reclist_slopeone = mySlopeOne.recommend("5", 10)

# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1080                           Celestial Clockwork (1994)
1653    Entertaining Angels: The Dorothy Day Story (1996)
1512               World of Apu, The (Apur Sansar) (1959)
1536                                 Aiqing wansui (1994)
1463                                     Boys, Les (1997)
1293                                      Star Kid (1997)
814                         Great Day in Harlem, A (1994)
112                                        Flipper (1996)
111                   Truth About Cats & Dogs, The (1996)
108                  Kids in the Hall: Brain Candy (1996)
Name: title, dtype: object

Se puede apreciar que hay aun más diferencia en las recomendaciones, lo que indica que el sistema es bastante personalizado.

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**

Se puede apreciar que las listas son completamente distintas. Se podría decir que las películas más populares no son las mejores evaluadas. De hecho, para las otras dos listas de recomendaciones de Slope One (usuarios 457 y 5) no se obtienen similitudes con las películas más populares.

In [36]:
# Definicion de objeto "most popular"
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

# Calcular las recomendaciones para un usuario en particular (id=44)
user_id = 44

top_n = 10

ranking_most_popular = [int(r) for r in most_popular.recommend(str(user_id), top_n, includeRated=False)]

# Lo convertimos a numpy array
recmovies_most_popular = np.array(ranking_most_popular).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Most Popular - Lista de items por nombre:')
info_file.loc[recmovies_most_popular]['title']

Most Popular - Lista de items por nombre:


movieid
50                     Star Wars (1977)
100                        Fargo (1996)
181           Return of the Jedi (1983)
286         English Patient, The (1996)
1                      Toy Story (1995)
121       Independence Day (ID4) (1996)
300                Air Force One (1997)
127               Godfather, The (1972)
7                 Twelve Monkeys (1995)
98     Silence of the Lambs, The (1991)
Name: title, dtype: object

In [37]:
# Mediante el método recommend() genereremos una lista top-10 recomendaciones para el usuario ID:44
reclist_slopeone = mySlopeOne.recommend("44", 10)

# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Slope One - Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Slope One - Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
Name: title, dtype: object